In [1]:
import cudf as pd
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
adv_data = pd.read_csv('data/adv_data.csv')
event_data = pd.read_csv('data/event_data.csv')
users_data = pd.read_csv('data/users_data.csv')

In [3]:
users_data.drop('delete_dtm', inplace=True, axis=1)

In [4]:
event_phones = event_data['user_phone'].to_arrow()

In [5]:
np_test = np.zeros_like(event_phones)

In [6]:
for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')

In [7]:
event_data['user_phone_edited'] = np_test

In [8]:
event_data.drop('user_phone', axis=1, inplace=True)

In [9]:
user_phones = users_data['user_phone'].to_arrow()

In [10]:
np_test = np.zeros_like(user_phones)

In [11]:
for i, val in enumerate(user_phones):
    np_test[i] = str(val).strip('+')

In [12]:
users_data['user_phone_edited'] = np_test

In [13]:
users_data.drop('user_phone', axis=1, inplace=True)

In [14]:
adv_data_plus_user_data = adv_data.merge(users_data, how='inner', on='user_id')

In [15]:
del adv_data, users_data

In [16]:
all_data = adv_data_plus_user_data.merge(event_data, how='inner', on='user_phone_edited')

In [24]:
del event_data

In [25]:
all_data.drop(['id', 'user_id'], inplace=True, axis=1)

In [33]:
for col in all_data.columns:
    print(col, ':')
    print(all_data[col].isna().sum())
    

campaign_type :
0
campaign_start_dtm :
0
campaign_end_dtm :
0
source_medium :
0
utm_campaign :
136003
interface :
0
currency_x :
0
campaign_cost :
10554952
registration_dtm :
0
user_phone_edited :
0
event_type :
0
event_dtm :
0
event_status :
12988351
update_dtm :
10392431
payout :
10555149
currency_y :
0
product_type :
0


In [34]:
all_data.head(50)

,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_x,campaign_cost,registration_dtm,user_phone_edited,event_type,event_dtm,event_status,update_dtm,payout,currency_y,product_type
0,show,2023-03-23 00:00:50.747478,2023-03-23 00:00:50.747478,email / email,light_market_mind,api,RUB,<NA>,2022-12-12 15:03:03.954903,79130534887,show,2023-06-09 08:19:02.530911,<NA>,<NA>,<NA>,RUB,loan
1,show,2023-06-26 04:44:18.951711,2023-06-26 04:44:18.951711,android / mobile_app,case_title_brandkeys,app,RUB,<NA>,2022-12-05 13:14:11.424364,79129133322,show,2024-01-25 15:09:28.858398,<NA>,<NA>,<NA>,RUB,loan
2,show,2023-06-07 14:21:16.186214,2023-06-07 14:21:16.186214,ios / mobile_app,megamaket_case_megamaket,app,RUB,<NA>,2022-12-05 13:14:11.424364,79129133322,show,2024-01-25 15:09:28.858398,<NA>,<NA>,<NA>,RUB,loan
3,show,2023-09-15 06:48:01.539898,2023-09-15 06:48:01.539898,email / email,adv_clickarrow_adv,web,RUB,<NA>,2023-02-12 03:17:37.428320,79151899946,show,2023-05-16 02:55:46.937274,<NA>,<NA>,<NA>,USD,loan
4,show,2023-10-31 01:12:46.952080,2023-10-31 01:12:46.952080,yandex / organic,target_megamaket_title,app,RUB,<NA>,2023-01-19 19:56:07.229785,79138592944,show,2023-06-08 00:13:07.800612,<NA>,<NA>,<NA>,RUB,loan
5,click,2023-02-13 08:50:47.699661,2023-02-13 08:50:47.699661,not-set / none,<NA>,api,RUB,0.0,2022-11-21 18:07:21.126740,79135523206,show,2023-09-15 08:48:04.880418,<NA>,<NA>,<NA>,RUB,loan
6,show,2023-01-09 02:17:46.850160,2023-01-09 02:17:46.850160,ya.ru / cpc,market_retarget_response,app,RUB,<NA>,2022-11-21 18:07:21.126740,79135523206,show,2023-09-15 08:48:04.880418,<NA>,<NA>,<NA>,RUB,loan
7,show,2023-04-28 11:00:55.919846,2023-04-28 11:00:55.919846,ya.ru / cpc,client_case_light,app,RUB,<NA>,2023-01-23 13:09:26.305631,79111827379,show,2023-11-10 22:25:29.097498,<NA>,<NA>,<NA>,RUB,mortgage
8,show,2023-10-20 05:31:50.368597,2023-10-20 05:31:50.368597,google / organic,clickarrow_target_brandkeys,web,USD,<NA>,2023-02-25 06:01:57.990335,79177704623,show,2024-01-15 07:06:57.046293,<NA>,<NA>,<NA>,RUB,loan
9,click,2023-09-24 01:07:08.513977,2023-09-24 01:07:08.513977,yahoo / organic,target_megamaket_clickarrow,web,RUB,0.0,2023-02-25 06:01:57.990335,79177704623,show,2024-01-15 07:06:57.046293,<NA>,<NA>,<NA>,RUB,loan


In [ ]:
# all_data.to_parquet('all_data.parquet')